In [30]:
import pandas as pd
import numpy as np
import requests
import uuid

In [2]:
url = "http://127.0.0.1:1234"
endpoint = "/v1/chat/completions"
api_url = url + endpoint

In [40]:
query  = ""
context = ""

In [27]:
def call_llm(payload):
    response = requests.post(api_url, json=payload).json()
    output = response["choices"][0]["message"]["content"]
    return output

In [36]:
# sample playbook instruction:

# Generate a random UUID version 4 (randomly generated)
uuid_value = str(uuid.uuid4())

counter_benefit = 0
counter_harm = 0

content = "When you open a door, you either have a bedroom, a bathroom, or a bigger space to enter into."

sample_playbook_rule = {
    uuid_value: {
        "counter_benefit": counter_benefit,
        "counter_harm": counter_harm,
        "content": content
    }
}

sample_playbook_rule

{'7a7d2218-9937-4c15-9157-0c2e9c2f6b25': {'counter_benefit': 0,
  'counter_harm': 0,
  'content': 'When you open a door, you either have a bedroom, a bathroom, or a bigger space to enter into.'}}

## Defining the Playbook

In [38]:
playbook = {
    uuid_value: {
        "counter_benefit": counter_benefit,
        "counter_harm": counter_harm,
        "content": content
    }
}

playbook

{'7a7d2218-9937-4c15-9157-0c2e9c2f6b25': {'counter_benefit': 0,
  'counter_harm': 0,
  'content': 'When you open a door, you either have a bedroom, a bathroom, or a bigger space to enter into.'}}

## Defining the Generator

In [76]:
def create_generator_prompt(query, context):
    generator_system_prompt = """
    You are an analysis expert tasked with answering questions using your knowledge, a curated playbook of strategies and insights and a
    reflection that goes over the diagnosis of all previous mistakes made while answering the question.

    Instructions: - Read the playbook carefully and apply relevant strategies, formulas, and insights - Pay attention to common mistakes
    listed in the playbook and avoid them - Show your reasoning step-by-step - Be concise but thorough in your analysis - If the playbook
    contains relevant code snippets or formulas, use them appropriately - Double-check your calculations and logic before providing the final
    answer

    Your output should be a json object, which contains the following fields: - reasoning: your chain of thought / reasoning / thinking process,
    detailed analysis and calculations - bullet_ids: each line in the playbook has a bullet_id. all bulletpoints in the playbook that’s relevant,
    helpful for you to answer this question, you should include their bullet_id in this list - final_answer: your concise final answer

    Avoid generating any thinking outputs. Just give the straight answer without any thinking.
    """

    generator_user_prompt = f"""
    Here is the following information you will require:
    Playbook:
    {playbook}

    Question:
    {query}

    Context:
    {context}

    Answer in this exact JSON format witht the following keys:
    "reasoning": "[Your chain of thought / reasoning / thinking process, detailed analysis and calculations]",
    "useful_bullet_ids": ["the bullet id corresponding to in the playbook that was the most useful this"],
    "misleading_bullet_ids": ["the bullet id corresponding to in the playbook that was not useful at all"],
    "final_answer": "[Your concise final answer here]"
    """

    return generator_system_prompt, generator_user_prompt

In [77]:
query = "tell me the path to my personal bedroom in a linkedlist format starting from the master bedroom."
context = """the house has 10 doors on the basement floor. it has 6 doors on the upper floor. 
            there are two bedrooms downstairs and two upstairs. 
            there are two bathrooms downstairs and two upstairs. 
            there is a stair connecting the basement and upper floor.
            the master room is on the basement floor. the door from the basement floor opens up the living room.
            the stair is in the living room. 
            there's another door by the kitchen that leads to the living room.
            the main door of the house leads to the living room.
            there is a bedroom in a corner and to the side of the other bedroom upstairs.
            the stairs leads up to a greater area upstairs.
            the stairs is near the ac control box.
            one of the bedrooms is near the ac control box.
            the ac control box is in the center of the upstairs.
             """

In [78]:
generator_system_prompt, generator_user_prompt = create_generator_prompt(query, context)

In [79]:
generator_payload = {
    "messages": [
        {
            "role": "system",
            "content": generator_system_prompt
        },
        {
            "role": "user",
            "content": generator_user_prompt
        }
    ]
}
generate_response = call_llm(generator_payload)

In [81]:
generator_final_output =  generate_response.split("</think>")[1]

In [ ]:
generator_final_output